In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import gurobi as gp
from scipy.optimize import minimize

It costs a company $12 to purchase an hour of labor and $15 to purchase an hour of capital. If L hours of labor and K units of capital are available, then $$(0.05)(L^{2/3})(K^{1/3})$$ machines can be produced. We will find the maximum number of machines the company can produce, given the company has $100,000 to purchase labor and capital.

In [2]:
# Objective function
def prod_capacity (LK):
    return -(0.05*(LK[0]**(2/3))*(LK[1]**(1/3)))

In [3]:
# Constraint functions
# Must be >= 0 if type is inequality 
# 100000 - 12*LK[0] - 15*LK[1] >= 0
def budget_con (LK):
    return 100000 - 12*LK[0] - 15*LK[1]

In [4]:
# now we create a dictionary telling the solver what type of constraint and what the function is
constr1 = {'type':'ineq', 'fun': budget_con}
# then we put all our constraint dictionaries into a list
constraints = [constr1]

In [5]:
# do not really need to give method, it will use the best method it thinks
prod_max = minimize(prod_capacity,[1, 1],constraints=constraints)

In [6]:
prod_max.x

array([5555.32176145, 2222.40925742])

In [7]:
-(prod_max.fun)

204.66841622595192

The file homework4stocks.csv contains historical monthly returns for 27 companies. The first row contains stock names, and the first column contains the dates. For each company, we calculate the estimated mean return and the estimated variance of return. Then we calculate the estimated correlations between the companies’ returns.

We will find a portfolio that achieves an expected monthly return of at least 1% and minimizes portfolio variance. We will find the optimal investment strategy such that we have the least variance. 

In [8]:
df = pd.read_csv('homework4stocks.csv')

In [9]:
# find the mean of each stock
mean_series = df.mean()
# find the variance of each stock
var_series = df.var()
# find the covariance of each stock
cov_df = df.cov()

/var/folders/03/7tjd273d5v3c9w_mbksz2h8r0000gn/T/ipykernel_20297/1970453582.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  mean_series = df.mean()
/var/folders/03/7tjd273d5v3c9w_mbksz2h8r0000gn/T/ipykernel_20297/1970453582.py:4: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  var_series = df.var()


In [10]:
cov_df

,AA,AAPL,AXP,BA,CAT,CSCO,DD,DIS,EK,FDX,...,KO,MCD,MMM,MO,MRK,MSFT,PG,T,UTX,WMT
AA,0.013656,0.006025,0.007063,0.004233,0.008318,0.004408,0.005365,0.004893,0.004586,0.004916,...,0.002070,0.001884,0.003601,0.001283,0.002432,0.003552,0.001582,0.002696,0.003637,0.001022
AAPL,0.006025,0.014295,0.006367,0.003505,0.004729,0.003312,0.003641,0.003477,0.002872,0.003340,...,0.001133,0.002767,0.002614,0.002288,0.001295,0.003710,0.001251,0.002266,0.002393,-0.000312
AXP,0.007063,0.006367,0.017277,0.005524,0.008176,0.005145,0.007190,0.005690,0.004734,0.006812,...,0.001420,0.001436,0.004510,0.001823,0.001134,0.004006,0.002385,0.002160,0.004179,0.000773
BA,0.004233,0.003505,0.005524,0.006585,0.003458,0.002636,0.003765,0.003279,0.005510,0.002633,...,0.000982,0.001156,0.002239,0.001244,0.002613,0.001521,0.001063,0.000953,0.003095,0.000043
CAT,0.008318,0.004729,0.008176,0.003458,0.010547,0.004454,0.004997,0.004118,0.003857,0.005635,...,0.002025,0.001793,0.003412,0.000787,0.001137,0.002747,0.002305,0.002742,0.003426,0.000914
CSCO,0.004408,0.003312,0.005145,0.002636,0.004454,0.006239,0.003343,0.003171,0.002940,0.003255,...,0.001462,0.000805,0.002396,0.000569,0.001451,0.002552,0.001373,0.001678,0.002034,0.000421
DD,0.005365,0.003641,0.007190,0.003765,0.004997,0.003343,0.005926,0.003772,0.003683,0.003924,...,0.001206,0.001271,0.002797,0.001567,0.001835,0.001855,0.001279,0.001226,0.002760,0.001033
DIS,0.004893,0.003477,0.005690,0.003279,0.004118,0.003171,0.003772,0.004637,0.002549,0.002931,...,0.001433,0.001527,0.002031,0.001324,0.001623,0.001985,0.001097,0.000952,0.002446,0.000640
EK,0.004586,0.002872,0.004734,0.005510,0.003857,0.002940,0.003683,0.002549,0.022754,0.002976,...,0.000805,0.002389,0.002289,0.001276,0.003146,0.003594,0.001283,0.001283,0.002866,0.001622
FDX,0.004916,0.003340,0.006812,0.002633,0.005635,0.003255,0.003924,0.002931,0.002976,0.006042,...,0.001197,0.001042,0.003122,0.000287,0.000301,0.001662,0.001617,0.001598,0.002240,0.001225


In [11]:
threshold_return = 0.01
# turn the mean series into an numpy array
mean_array = mean_series.values
# turn the covariance dataframe into a numpy array
cov_array = cov_df.values

In [12]:
# constraint0: weights sum to 1
# constraint1: mean return is >= 1%

# row 0 is all 1s already
A = np.ones((2,27)) 
# need mean return in row 1
A[1,:] = mean_array # now change row 1

# row 0 is already 1
b = np.ones(2) 
# mean return bigger than threshold
b[1] = threshold_return

In [13]:
portMod = gp.Model()
portMod_x = portMod.addMVar(27,ub=np.array([1]*27))
# can create constraints with different signs wihtout using addMConstr
weightsum_con = portMod.addConstr(A[0,:] @ portMod_x == b[0])
threshold_con = portMod.addConstr(A[1,:] @ portMod_x >= b[1])
# objective function is variance, w * cov(ri,rj) * w
portMod.setObjective(portMod_x @ cov_array @ portMod_x,sense=gp.GRB.MINIMIZE)

portMod.Params.OutputFlag = 0 

portMod.optimize()

Set parameter Username
Academic license - for non-commercial use only - expires 2024-03-23


In [14]:
np.sqrt(portMod.objVal)

0.02973382638840736

The file ‘variable_selection.csv’ contains observations of variables y, x1, x2, and x3. Here, y is the dependent variable. We want to choose a linear model that uses at most two independent variables such that the sum of squared residuals is minimized. We will formulate this problem as a constrained quadratic programming problem.

This is called the best subset problem, which is usually very hard to solve. We will solve this problem by enumeration. Specifically, we will run six OLS regressions (three with one independent variable and three more with two variables each) and choose the regression that best fits the data.

In [15]:
df3 = pd.read_csv('variable_selection.csv')
# drop the unnamed column
df3 = df3.drop(columns=['Unnamed: 0'])

In [16]:
df3.head()

,y,x1,x2,x3
0,42.412844,4.529686,7.172525,15.356190
1,24.820174,2.516030,4.377053,6.084709
2,26.617801,3.038386,4.411439,11.563392
3,21.329637,1.602293,4.225374,9.365461
4,34.183805,4.148982,5.618860,8.534010


In [17]:
from patsy import dmatrices
import statsmodels.api as sm
y, X = dmatrices('y ~ x1', data=df3, return_type='dataframe')

In [18]:
# Linear regression on the the three x variables
model = sm.OLS(y, X)
results = model.fit()
print(results.rsquared)

0.08354776627943716


In [19]:
from patsy import dmatrices
y, X = dmatrices('y ~ x2', data=df3, return_type='dataframe')

In [20]:
# Linear regression on the the three x variables
model = sm.OLS(y, X)
results = model.fit()
print(results.rsquared)

0.8980660003391546


In [21]:
from patsy import dmatrices
y, X = dmatrices('y ~ x3', data=df3, return_type='dataframe')

In [22]:
# Linear regression on the the three x variables
model = sm.OLS(y, X)
results = model.fit()
print(results.rsquared)

0.00533314436705179


In [23]:
from patsy import dmatrices
y, X = dmatrices('y ~ x1+x2', data=df3, return_type='dataframe')

In [24]:
# Linear regression on the the three x variables
model = sm.OLS(y, X)
results = model.fit()
print(results.rsquared)

0.996962185200456


In [25]:
from patsy import dmatrices
y, X = dmatrices('y ~ x1+x3', data=df3, return_type='dataframe')

In [26]:
# Linear regression on the the three x variables
model = sm.OLS(y, X)
results = model.fit()
print(results.rsquared)

0.08832769014424058


In [27]:
from patsy import dmatrices
y, X = dmatrices('y ~ x2+x3', data=df3, return_type='dataframe')

In [28]:
# Linear regression on the the three x variables
model = sm.OLS(y, X)
results = model.fit()
print(results.rsquared)

0.8981419458232157


The goal is to determine a set of ratings for the 32 NFL teams that most accurately predicts the actual outcomes of the games played. We use NLP to find the ratings that best predict the actual point spreads observed. The model will estimate the home team advantage and the ratings. The objective is to minimize the sum of squared prediction errors.

We will need to calculate the following:
* Actual Point Spread = Home Team Score – Visiting Team Score
* Predicted Spread = Home Team Rating – Visitor Team Rating + Home Team Advantage
* Prediction error = Actual Point Spread – Predicted Point Spread

We will also normalize the ratings. To do this, we set the actual average of the ratings to be 85 (this is somewhat arbitrary but based on the well-known Sagarin rating system).

In [29]:
# Don't use first row as column names
df4 = pd.read_csv('nflratings.csv',header=None)
# Drop the first column
df4 = df4.drop(columns=[0])
df4.shape
df4

,1,2,3,4
0,25,31,13,10
1,2,17,19,7
2,29,26,28,0
3,21,32,23,17
4,3,16,38,24
...,...,...,...,...
251,5,20,23,10
252,11,6,23,37
253,22,6,37,0
254,8,15,23,17


In [30]:
# scores matrix
score_spread = np.zeros((32,32))
for i in range(0,256):
  score_spread[df4[1].iloc[i]-1][df4[2].iloc[i]-1] = df4[3].iloc[i] - df4[4].iloc[i]

In [31]:
score_spread[0]

array([  0.,   0.,   0.,   0., -13.,   0.,   0.,   0.,   0.,   0.,   0.,
       -26.,   7., -21.,   0.,   0.,   0.,  13.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,  21.,   0.,  -4.,  11.,   0.,   0.,   0.])

In [32]:
# objective function
def least_squares (xy):
    sum_squared_diff = 0
    for i in range(len(score_spread)):
        for j in range(len(score_spread)):
            # if the score spread is not 0, then we have a game
            if score_spread[i][j] != 0:
                sum_squared_diff += (xy[i]-xy[j]+xy[32] - score_spread[i][j])**2
    return sum_squared_diff

In [33]:
# constraint where average rating is 85
def norm_con (xy):
    # exclude the constant, home field advantage
    return np.mean(xy[:32])-85

In [34]:
# now we create a dictionary telling the solver what type of constraint and what the function is
constr1 = {'type':'eq', 'fun': norm_con}
# then we put all our constraint dictionaries into a list
constraints = [constr1]

In [35]:
Input = np.full(33,1)
Input[32] = 1
Input

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [36]:
# do not really need to give method, it will use the best method it thinks
min_error = minimize(least_squares,Input,constraints=constraints)

rankings = min_error.x
rankings

array([84.52203291, 89.8789755 , 92.74623468, 83.08900292, 88.75984553,
       79.8135535 , 87.54744764, 76.88755002, 92.08546337, 85.63588776,
       70.50386423, 92.25163342, 86.98445096, 90.86230574, 78.4397591 ,
       76.88833122, 86.61528856, 92.06486032, 96.12269346, 95.62617207,
       85.09860842, 93.14847933, 75.03300878, 90.99555263, 86.64523408,
       67.67680456, 92.60329922, 85.91826039, 74.09773327, 79.1355281 ,
       82.18821619, 80.13392212,  2.12542539])

In [37]:
# how many correct?
correct = 0
for i in range(len(df4)):
    if (rankings[df4[1].iloc[i]-1] - rankings[df4[2].iloc[i]-1] + rankings[32]) > 0 and (df4[3].iloc[i] - df4[4].iloc[i]) > 0:
        correct += 1
    elif (rankings[df4[1].iloc[i]-1] - rankings[df4[2].iloc[i]-1] + rankings[32]) < 0 and (df4[3].iloc[i] - df4[4].iloc[i]) < 0:
        correct += 1

print(correct)

181


In [38]:
correct = 0
for i in range(32):
    for j in range(32):
        if score_spread[i][j] != 0:
            actual = score_spread[i][j]
            predicted = rankings[i] - rankings[j] + rankings[32]
            if actual > 0 and predicted > 0:
                correct += 1
            elif actual < 0 and predicted < 0:
                correct += 1
print(correct)

181
